In [3]:
# retrieve the raw data
import numpy as np  
import pandas as pd
import pickle
from connect_the_dots.io import get_CZI_metadata, get_CZI_zstack_timeseries
from connect_the_dots.filtering import wavelet_filter_zstack        
from connect_the_dots.tracking import get_localizations_iterative
from connect_the_dots.tracking import filter_large_displacements_in_trackpy_trajectories
from connect_the_dots.tracking import link_trajectories_across_channels
from connect_the_dots.tracking import infer_missing_dot_locations
from connect_the_dots.tracking import fill_missing_dot_localizations4
from connect_the_dots.tracking import dot_volume_container      
from connect_the_dots.io import make_movie
import os

In [4]:
num_skipped = 0
skipped_list = []
df = pd.read_csv('../config/samples.csv')
count = 0
doing = []
for idx in np.arange(len(df)-1,0,-1):
    group = df.iloc[idx]['group']
    filename = df.iloc[idx]['file_names']
    folder = df.iloc[idx]['file_paths']
    
    mov_file = os.path.join('../results',f"{group}",filename+'.movie.mp4')
    df_file = os.path.join('../results',f"{group}",filename+'.tracks.csv')
    
    # skip existing movie files
    if os.path.exists(mov_file)==True:
        continue
        
    # skip non-existing trajectory files
    if os.path.exists(df_file)==False:
        print()
#         print(f"Skipping movie {idx}")    
        print(f"{folder.split('/')[-1]}/{filename}")
        num_skipped+=1
        skipped_list.append(f"{folder.split('/')[-1]}/{filename}")
        continue   
    count+=1
    doing.append(filename)
print(count)
doing

32


['2021_06_25_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1.czi',
 '2021_06_28_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1-01.czi',
 '2021_07_01_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie2-01.czi',
 '2021_06_29_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1-01.czi',
 '2021_06_30_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie2-02.czi',
 '2021_06_30_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1-01.czi',
 '2021_06_24_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1-01.czi',
 '2021_07_01_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1-01.czi',
 '2021_06_25_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie2.czi',
 '2021_06_29_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_3

In [5]:
num_skipped = 0
skipped_list = []
df = pd.read_csv('../config/samples.csv')

for idx in np.arange(len(df)-1,0,-1):#range(len(df)):
    group = df.iloc[idx]['group']
    filename = df.iloc[idx]['file_names']
    folder = df.iloc[idx]['file_paths']
    
    mov_file = os.path.join('../results',f"{group}",filename+'.movie.mp4')
    df_file = os.path.join('../results',f"{group}",filename+'.tracks.csv')
    
    # skip existing movie files
    if os.path.exists(mov_file)==True:
        continue
        
    # skip non-existing trajectory files
    if os.path.exists(df_file)==False:
        print()
#         print(f"Skipping movie {idx}")    
        print(f"{folder.split('/')[-1]}/{filename}")
        num_skipped+=1
        skipped_list.append(f"{folder.split('/')[-1]}/{filename}")
        continue        
    
    print()
    print(f"Doing: {filename}")
    
    print("Loading tracks")    
    filled_df = pd.read_csv(df_file)    
    
    print("Loading metadata...")
    in_file = os.path.join(folder,filename)
    data_info, metadata = get_CZI_metadata(in_file)
    frames = range(data_info[3])        
    
    Z_dict = {}
    for ch in [0,1]:
        # get the image time series
        print("Loading image time series...")
        Z = get_CZI_zstack_timeseries(in_file,frames,ch)
        Z_dict[ch] = Z    
        
    # make movies
    print(f"Making raw movies and trajectories")
    make_movie(Z_dict,
           filled_df,
           output_filename=mov_file,
           metadata=metadata,
           desired_bar_length_um=10,
           percent_cutoff=99.99,
           disp_line_len=25,
           verbose=False,
           max_axis=0,
           text_dist=10,
           adaptive_text=False,
           line_alpha=0.5,
           adaptive_LUT=True,
           millisecond_per_frame=100)        
    
print(f"Number skipped = {num_skipped}")    
for f in sorted(skipped_list):
    print(f)


Doing: 2021_06_25_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1.czi
Loading tracks
Loading metadata...
Loading image time series...
Loading image time series...
Making raw movies and trajectories
sleeping 3 seconds

Doing: 2021_06_28_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1-01.czi
Loading tracks
Loading metadata...
Loading image time series...
Loading image time series...
Making raw movies and trajectories
sleeping 3 seconds

Doing: 2021_07_01_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie2-01.czi
Loading tracks
Loading metadata...
Loading image time series...
Loading image time series...
Making raw movies and trajectories
sleeping 3 seconds

Doing: 2021_06_29_Fbn2_WAPL_sixhoursIAA_488nm_0p5_561nm_0p05_SC8_2X_30z_250nm_780V_365T_20s_movie1-01.czi
Loading tracks
Loading metadata...
Loading image time series...
Loading image time series...
Making raw movies and trajectories
sleepi